In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json
"""
Your task is to wrangle the data and transform the shape of the data
into the model we mentioned earlier. The output should be a list of dictionaries
that look like this:

{
"id": "2406124091",
"type: "node",
"visible":"true",
"created": {
          "version":"2",
          "changeset":"17206049",
          "timestamp":"2013-08-03T16:43:42Z",
          "user":"linuxUser16",
          "uid":"1219059"
        },
"pos": [41.9757030, -87.6921867],
"address": {
          "housenumber": "5157",
          "postcode": "60625",
          "street": "North Lincoln Ave"
        },
"amenity": "restaurant",
"cuisine": "mexican",
"name": "La Cabana De Don Luis",
"phone": "1 (773)-271-5176"
}

You have to complete the function 'shape_element'.
We have provided a function that will parse the map file, and call the function with the element
as an argument. You should return a dictionary, containing the shaped data for that element.
We have also provided a way to save the data in a file, so that you could use
mongoimport later on to import the shaped data into MongoDB. 

Note that in this exercise we do not use the 'update street name' procedures
you worked on in the previous exercise. If you are using this code in your final
project, you are strongly encouraged to use the code from previous exercise to 
update the street names before you save them to JSON. 

In particular the following things should be done:
- you should process only 2 types of top level tags: "node" and "way"
- all attributes of "node" and "way" should be turned into regular key/value pairs, except:
    - attributes in the CREATED array should be added under a key "created"
    - attributes for latitude and longitude should be added to a "pos" array,
      for use in geospacial indexing. Make sure the values inside "pos" array are floats
      and not strings. 
- if the second level tag "k" value contains problematic characters, it should be ignored
- if the second level tag "k" value starts with "addr:", it should be added to a dictionary "address"
- if the second level tag "k" value does not start with "addr:", but contains ":", you can
  process it in a way that you feel is best. For example, you might split it into a two-level
  dictionary like with "addr:", or otherwise convert the ":" to create a valid key.
- if there is a second ":" that separates the type/direction of a street,
  the tag should be ignored, for example:

<tag k="addr:housenumber" v="5158"/>
<tag k="addr:street" v="North Lincoln Avenue"/>
<tag k="addr:street:name" v="Lincoln"/>
<tag k="addr:street:prefix" v="North"/>
<tag k="addr:street:type" v="Avenue"/>
<tag k="amenity" v="pharmacy"/>

  should be turned into:

{...
"address": {
    "housenumber": 5158,
    "street": "North Lincoln Avenue"
}
"amenity": "pharmacy",
...
}

- for "way" specifically:

  <nd ref="305896090"/>
  <nd ref="1719825889"/>

should be turned into
"node_refs": ["305896090", "1719825889"]
"""


lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]




In [109]:
nod = []
for _, element in ET.iterparse('prepare.osm'):
    if element.tag == 'node':
       ## for tag in element.iter('tag'):
       ##     print tag
        print element.items()

[('changeset', '11129782'), ('uid', '451048'), ('timestamp', '2012-03-28T18:31:23Z'), ('lon', '-87.6866303'), ('visible', 'true'), ('version', '7'), ('user', 'bbmiller'), ('lat', '41.9730791'), ('id', '261114295')]
[('changeset', '8448766'), ('uid', '451048'), ('timestamp', '2011-06-15T17:04:54Z'), ('lon', '-87.6878512'), ('visible', 'true'), ('version', '6'), ('user', 'bbmiller'), ('lat', '41.9730416'), ('id', '261114296')]
[('changeset', '8581395'), ('uid', '451048'), ('timestamp', '2011-06-29T14:14:14Z'), ('lon', '-87.6939548'), ('visible', 'true'), ('version', '5'), ('user', 'bbmiller'), ('lat', '41.9729565'), ('id', '261114299')]
[('changeset', '8581395'), ('uid', '451048'), ('timestamp', '2011-06-29T14:14:14Z'), ('lon', '-87.6976025'), ('visible', 'true'), ('version', '5'), ('user', 'bbmiller'), ('lat', '41.9707380'), ('id', '261146436')]
[('changeset', '8581395'), ('uid', '451048'), ('timestamp', '2011-06-29T14:14:15Z'), ('lon', '-87.6988576'), ('visible', 'true'), ('version', '

In [70]:
[] is None

False

In [110]:
result = []
for _, element in ET.iterparse('prepare.osm'):
    node = {}
    if element.tag == "node" or element.tag == "way" :
        
        if element.attrib.has_key('id'):
            node['id']= element.attrib['id']

        node['type'] = element.tag   
            
        if element.attrib.has_key('visible'):
            node['visible'] = element.attrib['visible'] == 'true'

        node['created'] = {}
        for key in CREATED:
            if element.attrib.has_key(key):
                node['created'][key] = element.attrib[key]
        
        if element.attrib.has_key('lat') and element.attrib.has_key('lon'):
            node['pos'] = [element.attrib['lat'], element.attrib['lon']]
        
        temp_dict = {}
        for tag in element.iter('tag'):
            st_1 = re.compile('.*:(.*)').findall(tag.attrib['k'])
            
            st_2 = re.compile('.*:.*:(.*)').findall(tag.attrib['k'])
            # print st_2
            if st_2 != []:
                continue
            elif st_1 != []:
                str_1 = st_1[0]
                temp_dict[str_1] = tag.attrib['v']
                # print str_1
            else:
                node[tag.attrib['k']] = tag.attrib['v']
        if temp_dict != {}:
            node['address'] = temp_dict
        
        if element.tag == 'way':
            node['node_refs'] = []
            for nd in element.iter('nd'):
                node['node_refs'].append(nd.attrib['ref'])
        result.append(node)
        
        
        
    

In [119]:
"result" == 'result'

True

In [141]:
def shape_element(element):
    node = {}
    if element.tag == "node" or element.tag == "way" :
        
        if element.attrib.has_key('id'):
            node['id']= element.attrib['id']

        node['type'] = element.tag   
            
        if element.attrib.has_key('visible'):
            node['visible'] = element.attrib['visible']

        node['created'] = {}
        for key in CREATED:
            if element.attrib.has_key(key):
                node['created'][key] = element.attrib[key]
        
        if element.attrib.has_key('lat') and element.attrib.has_key('lon'):
            node['pos'] = [float(element.attrib['lat']), float(element.attrib['lon'])]
        
        temp_dict = {}
        for tag in element.iter('tag'):
            st_1 = re.compile('.*:(.*)').findall(tag.attrib['k'])
            
            st_2 = re.compile('.*:.*:(.*)').findall(tag.attrib['k'])
            # print st_2
            if st_2 != []:
                continue
            elif st_1 != []:
                str_1 = st_1[0]
                temp_dict[str_1] = tag.attrib['v']
                # print str_1
            else:
                node[tag.attrib['k']] = tag.attrib['v']
        
        if temp_dict != {}:
            node['address'] = {}
            for k,v in temp_dict.items():
                if k == 'housenumber' or k == 'street':
                    node['address'][k] = temp_dict[k]
        
        if element.tag == 'way':
            node['node_refs'] = []
            for nd in element.iter('nd'):
                node['node_refs'].append(nd.attrib['ref'])
        
        
        return node
    else:
        return None


def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data



In [144]:
def test():
    # NOTE: if you are running this code on your computer, with a larger dataset, 
    # call the process_map procedure with pretty=False. The pretty=True option adds 
    # additional spaces to the output, making it significantly larger.
    data = process_map('prepare.osm', False)
    #pprint.pprint(data)
    
    correct_first_elem = {
        "id": "261114295", 
        "visible": "true", 
        "type": "node", 
        "pos": [41.9730791, -87.6866303], 
        "created": {
            "changeset": "11129782", 
            "user": "bbmiller", 
            "version": "7", 
            "uid": "451048", 
            "timestamp": "2012-03-28T18:31:23Z"
        }
    }
    print data[0]
    print correct_first_elem
    
    for k, v in data[0].items():
        if not correct_first_elem.has_key(k) or data[0][k] != correct_first_elem[k]:
            print k
    
    assert data[0] == correct_first_elem
    assert data[-1]["address"] == {
                                    "street": "West Lexington St.", 
                                    "housenumber": "1412"
                                      }
    assert data[-1]["node_refs"] == [ "2199822281", "2199822390",  "2199822392", "2199822369", 
                                    "2199822370", "2199822284", "2199822281"]
    print data[-1]
if __name__ == "__main__":
    test()

{'pos': [41.9730791, -87.6866303], 'visible': 'true', 'type': 'node', 'id': '261114295', 'created': {'uid': '451048', 'changeset': '11129782', 'version': '7', 'user': 'bbmiller', 'timestamp': '2012-03-28T18:31:23Z'}}
{'visible': 'true', 'created': {'changeset': '11129782', 'version': '7', 'user': 'bbmiller', 'timestamp': '2012-03-28T18:31:23Z', 'uid': '451048'}, 'type': 'node', 'id': '261114295', 'pos': [41.9730791, -87.6866303]}
{'building': 'yes', 'node_refs': ['2199822281', '2199822390', '2199822392', '2199822369', '2199822370', '2199822284', '2199822281'], 'created': {'uid': '674454', 'changeset': '15353317', 'version': '1', 'user': 'chicago-buildings', 'timestamp': '2013-03-13T15:58:04Z'}, 'visible': 'true', 'address': {'street': 'West Lexington St.', 'housenumber': '1412'}, 'type': 'way', 'id': '209809850'}


In [129]:
dic1 = {'pos': [41.9730791, -87.6866303], 'visible': 'true', 'type': 'node', 'id': '261114295', 'created': {'uid': '451048', 'changeset': '11129782', 'version': '7', 'user': 'bbmiller', 'timestamp': '2012-03-28T18:31:23Z'}}

In [130]:
dic2 = {'visible': 'true', 'created': {'changeset': '11129782', 'version': '7', 'user': 'bbmiller', 'timestamp': '2012-03-28T18:31:23Z', 'uid': '451048'}, 'type': 'node', 'id': '261114295', 'pos': [41.9730791, -87.6866303]}

In [131]:
dic1 == dic2

True